In [27]:
import pandas as pd
import numpy as np

In [28]:
import re
import nltk
from nltk.tokenize import word_tokenize

**Load Data**

In [29]:
df=pd.read_csv('train_data.csv')

In [30]:
df.head()

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


In [31]:
df.shape

(16990, 2)

In [32]:
df['label'].value_counts()

label
2     3545
18    2118
14    1822
9     1557
5      987
16     985
1      837
19     823
7      624
6      524
15     501
17     495
12     487
13     471
4      359
3      321
0      255
8      166
10      69
11      44
Name: count, dtype: int64

**Data Cleansing**

In [33]:
def cleansing(df):
    df_clean=df.str.lower()
    df_clean=[re.sub(r"\d+","",i )for i in df_clean]
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean]
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean]
    return df_clean

In [34]:
df['clean_text']=cleansing(df['text'])

In [35]:
df['clean_text'].iloc[1]

'buy las vegas sands as travel to singapore builds wells fargo says https t co flswicz'

In [36]:
df.head()

,text,label,clean_text
0,Here are Thursday's biggest analyst calls: App...,0,here are thursday s biggest analyst calls appl...
1,Buy Las Vegas Sands as travel to Singapore bui...,0,buy las vegas sands as travel to singapore bui...
2,"Piper Sandler downgrades DocuSign to sell, cit...",0,piper sandler downgrades docusign to sell citi...
3,"Analysts react to Tesla's latest earnings, bre...",0,analysts react to tesla s latest earnings brea...
4,Netflix and its peers are set for a ‘return to...,0,netflix and its peers are set for a return to ...


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

**Split train test**

In [38]:
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size = 0.2, random_state = 42,stratify=df['label'])

In [39]:
x_train.head()

5922     it s ecb rate decision day here s what to expe...
13498    twitter users were quick to spot liz truss see...
4517     jetblue announces webcast of second quarter ea...
16161     calm cal maine foods stock ticks higher on re...
1745     tower semiconductor and cadence expand collabo...
Name: clean_text, dtype: object

**Tokenize the words in the train and test data**

In [40]:
word_token_train = [word_tokenize(i) for i in x_train]
word_token_test = [word_tokenize(i) for i in x_test]

In [41]:
word_token_train[0:1]

[['it',
  's',
  'ecb',
  'rate',
  'decision',
  'day',
  'here',
  's',
  'what',
  'to',
  'expect',
  'via',
  'weberalexander',
  'amp',
  'carolynnlook',
  'https',
  't',
  'co',
  'isqgdue']]

**Remove Stop word**

In [42]:
from nltk.corpus import stopwords
nltk.download('stopwords')
# Get a list of stop words in the Indonesian language
stop_words = set(stopwords.words('english'))

# Display the top 20 stop words
list(stop_words)[:20]

[nltk_data] Downloading package stopwords to /home/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['few',
 'in',
 'each',
 'doing',
 'above',
 "it's",
 'being',
 'ain',
 'couldn',
 'below',
 'will',
 "should've",
 'we',
 'such',
 "didn't",
 't',
 'had',
 'and',
 "mightn't",
 'from']

In [43]:
# Remove stopwords from each sublist in word_token_train and test
filtered_tokens_train = [[word for word in sublist if word not in stop_words] for sublist in word_token_train]
filtered_tokens_test = [[word for word in sublist if word not in stop_words] for sublist in word_token_test]


In [44]:
filtered_tokens_train[0:2]

[['ecb',
  'rate',
  'decision',
  'day',
  'expect',
  'via',
  'weberalexander',
  'amp',
  'carolynnlook',
  'https',
  'co',
  'isqgdue'],
 ['twitter',
  'users',
  'quick',
  'spot',
  'liz',
  'truss',
  'seemingly',
  'recreating',
  'outfit',
  'margaret',
  'thatcher',
  'appearance',
  'channel',
  'tory',
  'leadership',
  'debate',
  'https',
  'co',
  'vsiioegrz']]

**Text representation and Feature**

In [45]:
import gensim
from gensim.models import Word2Vec

# Skipgram
model_skipgram = gensim.models.Word2Vec(filtered_tokens_train, min_count = 3, vector_size = 50, window = 5, sg=1)

In [46]:
# Get word vectors
def get_word_vector(model, word):
    try:
        return model.wv.get_vector(word)
    except KeyError:
        # If the word is not in the model, return None
        return None

In [47]:
# Generate word vectors for filtered tokens using the specified word embedding model
def generate_word_vectors(filtered_tokens, word_vector_model, vector_length=50):
    # Initialize an empty list to store the resulting word vectors
    X_vectors = []

    # Iterate through each list of filtered tokens in the dataset
    for tokens in filtered_tokens:
        # Use the get_word_vector function to retrieve word vectors for each token
        vectorized_tokens = [get_word_vector(word_vector_model, word) for word in tokens]

         # Remove None values (words not present in the model) from the list of vectors
        vectorized_tokens = [vector for vector in vectorized_tokens if vector is not None]

        # If there are valid vectors, compute the average vector
        if vectorized_tokens:
            average_vector = sum(vectorized_tokens) / len(vectorized_tokens)
            X_vectors.append(average_vector)
        else:
            # If no valid vectors are present, use a zero vector as a placeholder
            X_vectors.append([0] * vector_length)

    # Return the list of resulting word vectors
    return X_vectors

In [48]:
# Transforming tokens in x_test into vectors using Skipgram model
x_train_vectors_skipgram = generate_word_vectors(filtered_tokens_train, model_skipgram)
x_test_vectors_skipgram = generate_word_vectors(filtered_tokens_test, model_skipgram)

In [49]:
x_train_vectors_skipgram[0:2]

[array([-0.21777509,  0.33876237,  0.01099326,  0.1113826 ,  0.08266913,
        -0.07282944,  0.4259012 ,  0.4628457 , -0.20448656, -0.12639934,
         0.35003325, -0.41161633, -0.1057011 , -0.06293263, -0.22289094,
        -0.13418542,  0.20471711, -0.23892476, -0.4331851 , -0.06881914,
         0.29489204,  0.3629736 ,  0.33743995, -0.42009962,  0.2222672 ,
         0.12430245,  0.09940781,  0.10020967, -0.20665008, -0.21086837,
        -0.41958436,  0.11416674, -0.21859255,  0.13192485, -0.32897013,
         0.32303306,  0.0598074 ,  0.0521929 ,  0.03762154,  0.12482833,
         0.31350765,  0.01794507, -0.00794016, -0.03600593,  0.49034512,
        -0.03510183, -0.3053745 , -0.37916672,  0.30153516,  0.34918052],
       dtype=float32),
 array([-0.1807398 ,  0.47539517,  0.12666127, -0.23944601,  0.05878483,
        -0.2222887 ,  0.39145017,  0.5588    , -0.30069223, -0.14633623,
         0.21032003, -0.22512516, -0.28908017, -0.119987  , -0.21178724,
        -0.16375598, -0.013

**SVM for text classification**

In [50]:
from sklearn import svm
svm_class = svm.LinearSVC( random_state=42)
svm_class.fit(x_train_vectors_skipgram, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,42


**Testing**

In [51]:
test_svm_class=svm_class.predict(x_test_vectors_skipgram)

In [52]:
print('\nClassification Report\n')
print(classification_report(y_test, test_svm_class, target_names=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']))


Classification Report

              precision    recall  f1-score   support

           0       0.83      0.10      0.18        51
           1       0.69      0.66      0.67       167
           2       0.57      0.79      0.66       709
           3       0.25      0.02      0.03        64
           4       0.94      0.92      0.93        72
           5       0.83      0.95      0.89       198
           6       0.71      0.71      0.71       105
           7       0.83      0.68      0.75       125
           8       0.75      0.27      0.40        33
           9       0.47      0.31      0.38       311
          10       0.00      0.00      0.00        14
          11       0.00      0.00      0.00         9
          12       0.72      0.54      0.62        97
          13       0.00      0.00      0.00        94
          14       0.57      0.70      0.63       364
          15       0.67      0.46      0.54       100
          16       0.72      0.81      0.76       197
   

/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha